<a href="https://colab.research.google.com/github/Sakibapon/Deep-Learning/blob/master/CNN_Model_and_Optimize_using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install -U keras-tuner

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mist.load_data()

In [0]:
train_images=train_images/255.0
test_images=test_images/255.0

In [11]:
train_images[0].shape

(28, 28)

In [12]:
len(train_images)

60000

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [21]:
train_images.shape

(60000, 28, 28, 1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [28]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3038 - accuracy: 0.0986 - val_loss: 2.3024 - val_accuracy: 0.1055
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3039 - accuracy: 0.0996 - val_loss: 2.3063 - val_accuracy: 0.0925
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3039 - accuracy: 0.1018 - val_loss: 2.3045 - val_accuracy: 0.0985


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.7655 - accuracy: 0.7171 - val_loss: 0.5851 - val_accuracy: 0.7747
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.5614 - accuracy: 0.7929 - val_loss: 0.4966 - val_accuracy: 0.8202
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.5029 - accuracy: 0.8161 - val_loss: 0.4949 - val_accuracy: 0.8172


Epoch 1/3
1688/1688 [==============================] - 6s 3ms/step - loss: 2.3044 - accuracy: 0.0989 - val_loss: 2.3032 - val_accuracy: 0.1027
Epoch 2/3
1688/1688 [==============================] - 5s 3ms/step - loss: 2.3040 - accuracy: 0.0995 - val_loss: 2.3054 - val_accuracy: 0.1008
Epoch 3/3
1688/1688 [==============================] - 5s 3ms/step - loss: 2.3040 - accuracy: 0.1014 - val_loss: 2.3028 - val_accuracy: 0.1008


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3044 - accuracy: 0.0986 - val_loss: 2.3064 - val_accuracy: 0.0942
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3038 - accuracy: 0.1009 - val_loss: 2.3067 - val_accuracy: 0.0925
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3041 - accuracy: 0.0996 - val_loss: 2.3045 - val_accuracy: 0.1050


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 2.3042 - accuracy: 0.1007 - val_loss: 2.3036 - val_accuracy: 0.0973
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 2.3041 - accuracy: 0.1004 - val_loss: 2.3043 - val_accuracy: 0.1027
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 2.3039 - accuracy: 0.1012 - val_loss: 2.3062 - val_accuracy: 0.0942


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        27680     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                991296    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,022,122
Trainable params: 1,022,122
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.5028 - accuracy: 0.8169 - val_loss: 0.4711 - val_accuracy: 0.8252
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4684 - accuracy: 0.8300 - val_loss: 0.4977 - val_accuracy: 0.8138
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4467 - accuracy: 0.8404 - val_loss: 0.4444 - val_accuracy: 0.8405
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4262 - accuracy: 0.8490 - val_loss: 0.4115 - val_accuracy: 0.8500
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4092 - accuracy: 0.8543 - val_loss: 0.4174 - val_accuracy: 0.8470
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3971 - accuracy: 0.8556 - val_loss: 0.4194 - val_accuracy: 0.8473
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3844 - accuracy: 0.8612 - val_loss: 0.3832 - val_accuracy